<a href="https://colab.research.google.com/github/HARSHIT097/PYTORCH-TENSORFLOW_LEARN/blob/main/VGG_vs_Custom_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic - warmup

In [9]:
def foo():
  x = 'Python'
  def bar():
    x = 'Java'
  bar() 
  return x

print(foo())

Python


In [10]:
tuple1=(25)
print(tuple1)
print(type(tuple1))

25
<class 'int'>


In [12]:
def t (a, b=10):
  print(a ,b) 
t(3) 


3 10


In [13]:
a = 1
b = 1
k = a+b
k = k + True
k

3

In [14]:
var = 10
def foo():
  var = 15
foo()
print(var)

10


In [15]:
import numpy as np

percentiles = [98, 76.37, 55.55, 69, 88]
first_subject = np.array(percentiles)
print(first_subject.dtype)

float64


In [16]:
import numpy as np
series = [[23,45,12,679], [14,48,69,38]]
new_series = np.array(series)
print(new_series.ndim)
print(new_series.shape)
print(new_series.dtype)

2
(2, 4)
int64


In [17]:
from numpy import random 
dataset = np.array(['paul', 'jacob', 'vince', 'paul', 'miky', 'jacob', 'warren'])
new_var = (dataset == 'paul') | (dataset == 'jacob')
print(new_var)

[ True  True False  True False  True False]


#To Understand the model:

A CNN can be divided into two main parts: Feature learning and classification.

##Feature Learning
In this part, the main goal of the NN is to find patterns in the pixels of the images that can be useful to identify the targets of the classification. That happens in the convolution layers of the network that specializes in those patterns for the problem at hand.

##Classification
Now we want to use those patterns to classify our images to their correct label. This part of the network does exactly that job, it uses the inputs from the previous layers to find the best class to your matched patterns in the new image.



In [1]:
import tensorflow as tf


In [2]:
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

58892288/58889256 [==============================] - 0s 0us/step


In [4]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [5]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [7]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])


Epoch 1/50
65/65 [==============================] - 40s 117ms/step - loss: 1.7778 - accuracy: 0.4710 - val_loss: 1.1123 - val_accuracy: 0.5953
Epoch 2/50
65/65 [==============================] - 5s 81ms/step - loss: 0.9527 - accuracy: 0.6720 - val_loss: 0.9609 - val_accuracy: 0.6401
Epoch 3/50
65/65 [==============================] - 5s 83ms/step - loss: 0.6749 - accuracy: 0.7557 - val_loss: 0.9511 - val_accuracy: 0.6946
Epoch 4/50
65/65 [==============================] - 5s 83ms/step - loss: 0.4595 - accuracy: 0.8311 - val_loss: 0.9821 - val_accuracy: 0.6965
Epoch 5/50
65/65 [==============================] - 5s 83ms/step - loss: 0.3361 - accuracy: 0.8740 - val_loss: 0.9136 - val_accuracy: 0.7160
Epoch 6/50
65/65 [==============================] - 5s 81ms/step - loss: 0.2554 - accuracy: 0.9080 - val_loss: 1.0250 - val_accuracy: 0.7043
Epoch 7/50
65/65 [==============================] - 5s 80ms/step - loss: 0.1967 - accuracy: 0.9294 - val_loss: 1.0290 - val_accuracy: 0.7043
Epoch 8/50


#comparing to hand-made model

In [8]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling


hand_made_model = Sequential()
hand_made_model.add(Rescaling(1./255, input_shape=(150,150,3)))

hand_made_model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
hand_made_model.add(layers.MaxPooling2D(3))

hand_made_model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
hand_made_model.add(layers.MaxPooling2D(2))

hand_made_model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
hand_made_model.add(layers.MaxPooling2D(2))

hand_made_model.add(layers.Flatten())
hand_made_model.add(layers.Dense(50, activation='relu'))
hand_made_model.add(layers.Dense(20, activation='relu'))
hand_made_model.add(layers.Dense(5, activation='softmax'))


hand_made_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

hand_made_model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 4s 26ms/step - loss: 1.5387 - accuracy: 0.3163 - val_loss: 1.4218 - val_accuracy: 0.3288
Epoch 2/50
65/65 [==============================] - 1s 21ms/step - loss: 1.3659 - accuracy: 0.3927 - val_loss: 1.3116 - val_accuracy: 0.4669
Epoch 3/50
65/65 [==============================] - 1s 22ms/step - loss: 1.2732 - accuracy: 0.4506 - val_loss: 1.2821 - val_accuracy: 0.4494
Epoch 4/50
65/65 [==============================] - 1s 21ms/step - loss: 1.2371 - accuracy: 0.4642 - val_loss: 1.2434 - val_accuracy: 0.5233
Epoch 5/50
65/65 [==============================] - 1s 22ms/step - loss: 1.1417 - accuracy: 0.5168 - val_loss: 1.1909 - val_accuracy: 0.5195
Epoch 6/50
65/65 [==============================] - 1s 21ms/step - loss: 1.0642 - accuracy: 0.5562 - val_loss: 1.1717 - val_accuracy: 0.5350
Epoch 7/50
65/65 [==============================] - 1s 22ms/step - loss: 0.9702 - accuracy: 0.6214 - val_loss: 1.1191 - val_accuracy: 0.5525
Epoch 8/50
65

#conclusion

The VGG model is performing better than our custom model.